# Generate UMAP coordinates for each plate

## Import libraries

In [1]:
import glob
import pathlib
import pandas as pd
import umap

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features


## Set constants

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)


## Create list of paths to feature selected data per plate

In [3]:
# Set input paths
data_dir = pathlib.Path("../4.preprocess_features/data/single_cell_profiles")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
fs_files


['../4.preprocess_features/data/single_cell_profiles/slide4_sc_feature_selected.parquet',
 '../4.preprocess_features/data/single_cell_profiles/slide1_sc_feature_selected.parquet',
 '../4.preprocess_features/data/single_cell_profiles/slide2_sc_feature_selected.parquet',
 '../4.preprocess_features/data/single_cell_profiles/slide3_sc_feature_selected.parquet']

In [4]:
# Load feature data into a dictionary, keyed on plate name
cp_dfs = {x.split("/")[-1]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]


dict_keys(['slide4_sc_feature_selected.parquet', 'slide1_sc_feature_selected.parquet', 'slide2_sc_feature_selected.parquet', 'slide3_sc_feature_selected.parquet'])


[(63359, 285), (69631, 284), (51791, 290), (56139, 290)]

## Generate UMAP coordinates for each plate

**Note:** Only metadata that is common between plates are included in final data frame.

In [5]:
desired_columns = [
    "Metadata_Plate",
    "Metadata_Well",
    "Metadata_Site",
    "Metadata_CellLine",
    "Metadata_Condition",
    "Metadata_Nuclei_Site_Count",
    "Metadata_Nuclei_Location_Center_X",
    "Metadata_Nuclei_Location_Center_Y"
]

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    plate_name = pathlib.Path(plate).stem
    print("UMAP embeddings being generated for", plate_name)

    # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(random_state=umap_random_seed, n_components=umap_n_components)

    # Make sure NA columns have been removed
    cp_df = cp_dfs[plate]
    cp_df = feature_select(cp_df, operation="drop_na_columns", na_cutoff=0)

    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)
    filtered_meta_features = [
        feature for feature in meta_features if feature in desired_columns
    ]

    # Fit UMAP and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.fit_transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
    )
    print(embeddings.shape)

    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat(
        [cp_df.loc[:, filtered_meta_features].reset_index(drop=True), embeddings],
        axis=1,
    )

    # randomize the rows of the dataframe to plot the order of the data evenly
    cp_umap_with_metadata_df = cp_umap_with_metadata_df.sample(frac=1, random_state=0)

    # Generate output file and save
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate_name}.tsv")
    cp_umap_with_metadata_df.to_csv(output_umap_file, index=False, sep="\t")

UMAP embeddings being generated for slide4_sc_feature_selected


/home/jenna/mambaforge/envs/python_nuclear_speckle_env/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(63359, 2)
UMAP embeddings being generated for slide1_sc_feature_selected


/home/jenna/mambaforge/envs/python_nuclear_speckle_env/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(69631, 2)
UMAP embeddings being generated for slide2_sc_feature_selected


/home/jenna/mambaforge/envs/python_nuclear_speckle_env/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(51791, 2)
UMAP embeddings being generated for slide3_sc_feature_selected


/home/jenna/mambaforge/envs/python_nuclear_speckle_env/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(56139, 2)


In [6]:
# Print an example output file
print(cp_umap_with_metadata_df.shape)
cp_umap_with_metadata_df.head(10)

(56139, 10)


,Metadata_CellLine,Metadata_Condition,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,UMAP0,UMAP1
43949,293T,untreated,slide3,B4,M42,421,308.964551,1890.887057,-2.114363,3.254997
35655,293T,untreated,slide3,B4,M24,639,1125.333520,1432.788117,-0.947957,2.625839
8408,786O,ALY kd8,slide3,A2,M33,130,2112.869369,717.921421,0.239094,7.681274
30843,293T,untreated,slide3,B4,M14,717,397.793005,1145.036403,-2.496449,4.571123
675,786O,NTC,slide3,A1,M30,59,1336.285529,889.204939,0.215084,6.979181
55399,293T,untreated,slide3,B4,M8,641,575.693267,1700.470324,2.181890,5.424494
21558,786O,FIBP kd6,slide3,B3,M23,143,1534.953558,1177.834457,-1.208824,8.647558
37987,293T,untreated,slide3,B4,M2,590,43.840716,908.849233,-3.119766,6.357724
20005,786O,FIBP kd6,slide3,B3,M12,269,1640.430453,1253.972840,-0.125468,7.274877
4283,786O,ALY kd5,slide3,B1,M38,189,1356.529801,153.955850,-1.602567,9.233997


In [7]:
# Sort the DataFrame by Metadata_Nuclei_Site_Count in ascending order
sorted_df = cp_umap_with_metadata_df.sort_values(by='Metadata_Nuclei_Site_Count')

# Print the shape of the sorted DataFrame
print(sorted_df.shape)

# Display the first 10 rows of the sorted DataFrame
sorted_df.head(10)


(56139, 10)


,Metadata_CellLine,Metadata_Condition,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,UMAP0,UMAP1
28138,786O,FIBP kd7,slide3,A4,M9,17,1783.484642,1988.353053,-1.774260,10.679827
28132,786O,FIBP kd7,slide3,A4,M9,17,1965.260243,1205.643399,1.684780,8.476129
28136,786O,FIBP kd7,slide3,A4,M9,17,1940.582454,1771.015831,1.447367,9.626948
28127,786O,FIBP kd7,slide3,A4,M9,17,125.435390,424.052969,-0.462147,8.101818
28125,786O,FIBP kd7,slide3,A4,M9,17,1089.466887,111.100993,-1.887630,9.233392
28129,786O,FIBP kd7,slide3,A4,M9,17,1761.739187,583.457715,-2.074991,9.082572
28124,786O,FIBP kd7,slide3,A4,M9,17,1054.333752,103.191343,-2.149884,10.185878
28137,786O,FIBP kd7,slide3,A4,M9,17,1831.633380,1842.282123,1.784333,8.360373
28131,786O,FIBP kd7,slide3,A4,M9,17,1972.298442,1158.654015,-0.570769,7.662328
28128,786O,FIBP kd7,slide3,A4,M9,17,1721.281372,528.883866,1.746869,7.929691


In [8]:
# Filter rows where UMAP1 column has values above 10
filtered_df = cp_umap_with_metadata_df[cp_umap_with_metadata_df['UMAP1'] < 0]

# Print the shape of the filtered DataFrame
print(filtered_df.shape)

# Display the first 10 rows of the filtered DataFrame
filtered_df


(1461, 10)


,Metadata_CellLine,Metadata_Condition,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,UMAP0,UMAP1
48793,293T,untreated,slide3,B4,M51,742,26.935793,1039.465683,0.953574,-0.805659
48910,293T,untreated,slide3,B4,M51,742,834.527586,1337.311111,1.433411,-0.837687
48344,293T,untreated,slide3,B4,M50,716,1309.862834,1778.361573,1.241416,-1.258568
48770,293T,untreated,slide3,B4,M51,742,2158.086116,975.009959,0.933222,-1.057680
48201,293T,untreated,slide3,B4,M50,716,715.262480,1289.020129,0.916584,-0.689143
...,...,...,...,...,...,...,...,...,...,...
48682,293T,untreated,slide3,B4,M51,742,154.666096,711.233733,0.958546,-0.701799
47883,293T,untreated,slide3,B4,M50,716,963.337247,351.918890,1.319508,-0.566928
53887,293T,untreated,slide3,B4,M63,45,600.448772,58.116003,0.654928,-0.876581
47954,293T,untreated,slide3,B4,M50,716,352.030374,543.825935,0.900874,-0.722929


In [9]:
# Group by Metadata_Well and Metadata_Site and count the number of unique combinations
combo_counts = filtered_df.groupby(['Metadata_Well', 'Metadata_Site']).size().reset_index(name='counts')

# Display the resulting DataFrame with counts
print(combo_counts)


  Metadata_Well Metadata_Site  counts
0            B1           M63       1
1            B4           M46       2
2            B4           M50     671
3            B4           M51     670
4            B4           M52       1
5            B4           M62      76
6            B4           M63      40


In [10]:
# Filter rows where UMAP1 column has values above 10
filtered_df = cp_umap_with_metadata_df[cp_umap_with_metadata_df['UMAP0'] > 5]

# Print the shape of the filtered DataFrame
print(filtered_df.shape)

# Display the first 10 rows of the filtered DataFrame
filtered_df


(363, 10)


,Metadata_CellLine,Metadata_Condition,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,UMAP0,UMAP1
53867,293T,untreated,slide3,B4,M62,90,930.868282,446.132771,7.086971,2.540568
53922,293T,untreated,slide3,B4,M63,45,1427.816412,2064.692629,7.208066,2.739977
19231,786O,SART1 kd6,slide3,A3,M58,90,1818.941437,1523.096273,5.028321,2.758001
31964,293T,untreated,slide3,B4,M17,310,328.670507,163.165899,7.145517,2.624620
27547,786O,FIBP kd7,slide3,A4,M52,74,1299.557364,600.157364,7.288156,2.752332
...,...,...,...,...,...,...,...,...,...,...
26763,786O,FIBP kd7,slide3,A4,M37,82,1843.907289,806.887473,7.276399,2.732289
13392,786O,SART1 kd4,slide3,B2,M26,104,1334.438395,211.673352,5.272272,2.539229
23310,786O,FIBP kd6,slide3,B3,M38,187,1420.386286,1225.422857,7.506048,2.889270
23605,786O,FIBP kd6,slide3,B3,M42,67,1273.569919,1119.302439,7.280236,2.733738


In [11]:
# Group by Metadata_Well and Metadata_Site and count the number of unique combinations
combo_counts = filtered_df.groupby(['Metadata_Well', 'Metadata_Site']).size().reset_index(name='counts')

# Display the resulting DataFrame with counts
print(combo_counts)

    Metadata_Well Metadata_Site  counts
0              A1           M14       1
1              A1           M15       1
2              A1           M26       1
3              A1           M28       1
4              A1           M30       1
..            ...           ...     ...
197            B4           M60       2
198            B4           M61       3
199            B4           M62       3
200            B4           M63       1
201            B4           M64       2

[202 rows x 3 columns]
